<a href="https://colab.research.google.com/github/nagae/ICL_B_2021/blob/main/duration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# jp_df から特定の期間内のデータだけをとりだす
`slice` 関数を使うと，特定の期間内のデータだけを取り出すことができる.

In [1]:
# 必要なモジュールのインポート
import numpy as np # 行列計算
import pandas as pd # データベース
from matplotlib import pyplot as plt, dates as mdates # グラフ描画
import datetime # 日付
# ファイルを読み込み，列名を使いやすい形に修正
fname = 'https://toyokeizai.net/sp/visual/tko/covid19/csv/prefectures.csv'
jp_df = pd.read_csv(fname, dtype=str)
jp_df.columns = [ "年", "月" , "日", "都道府県", "Pref", "陽性数", "検査数", "入院数", "重症数", "退院数", "死亡数", "実効再生産数"]
# 都道府県名以外の全てのセルを整数値に変換する
for c in jp_df.columns:
    if c in ["都道府県", "Pref"]: # 都道府県名のデータは文字型として扱う
        jp_df[c] = jp_df[c].astype(str)
    else:
        jp_df[c] = pd.to_numeric(jp_df[c], errors='coerce') # 都道府県名以外のデータは数値として扱う
        jp_df[c] = jp_df[c].fillna(0) # データが欠損している部分を0で埋める
        jp_df[c] = jp_df[c].astype(float) # 一旦は実数型に変換する 
        if c not in ["実効再生産数"]: # 実効再生産数以外のデータは整数型として扱う
            jp_df[c] = jp_df[c].astype(int)

# 年月日の列からdatetime型のインデックスを生成する
jp_df["年月日"] = pd.to_datetime(jp_df[["年", "月", "日"]].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d')
# 都道府県名と年月日からなるマルチインデックスを構築
jp_df = jp_df.set_index(["都道府県", "年月日"])
jp_df # jp_df の概観を表示

,,年,月,日,Pref,陽性数,検査数,入院数,重症数,退院数,死亡数,実効再生産数
都道府県,年月日,,,,,,,,,,,
東京都,2020-02-08,2020,2,8,Tokyo,3,0,0,0,0,0,0.00
神奈川県,2020-02-08,2020,2,8,Kanagawa,1,0,0,0,0,0,0.00
東京都,2020-02-09,2020,2,9,Tokyo,3,0,0,0,0,0,0.00
神奈川県,2020-02-09,2020,2,9,Kanagawa,1,0,0,0,0,0,0.00
東京都,2020-02-10,2020,2,10,Tokyo,3,0,0,0,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
熊本県,2021-05-19,2021,5,19,Kumamoto,5485,74137,912,21,4484,89,0.98
大分県,2021-05-19,2021,5,19,Oita,3011,134662,661,6,2313,37,0.90
宮崎県,2021-05-19,2021,5,19,Miyazaki,2887,38333,354,5,2499,25,0.54


In [9]:
# 2つの日付を指定
date0 = '2021-04-01'
date1 = '2021-05-05'
jp_df2 = jp_df.xs(slice(date0, date1), level='年月日') # date0からdate1までの
jp_df2

,年,月,日,Pref,陽性数,検査数,入院数,重症数,退院数,死亡数,実効再生産数
都道府県,,,,,,,,,,,
北海道,2021,4,1,Hokkaido,21051,451342,802,17,19539,752,1.02
青森県,2021,4,1,Aomori,1039,25237,135,0,884,20,2.33
岩手県,2021,4,1,Iwate,646,45250,62,0,554,30,1.86
宮城県,2021,4,1,Miyagi,6103,91791,1490,9,4582,31,1.16
秋田県,2021,4,1,Akita,285,7580,7,0,272,6,1.00
...,...,...,...,...,...,...,...,...,...,...,...
熊本県,2021,5,5,Kumamoto,4324,66663,502,16,3742,80,0.87
大分県,2021,5,5,Oita,2099,114417,504,1,1567,28,0.99
宮崎県,2021,5,5,Miyazaki,2246,30480,173,0,2043,22,0.67
